In [2]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers
import os
from pathlib import Path
from operator import itemgetter

In [3]:
parent_dir = Path(os.path.join(os.getcwd(), os.pardir)).parent.parent
user_query = "what is Applied Generative AI for Beginners?"

In [4]:

src_dir = parent_dir.parent

model_dir = os.path.join(src_dir, 'model')
model_path=os.path.join(model_dir, 'llama-2-7b-chat.ggmlv3.q4_0.bin')

llm=CTransformers(
        model=model_path,
        model_type="llama",
        config={'max_new_tokens':128,
                'temperature':0.01,
                'context_length': 200
                }
)


# print(llm.invoke(user_query))

c:\Users\LENOVO\sagar\work\AI_ML\gitrepo\GENAI\YOUTUBES\GenAI_research\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



 Unterscheidung between Generative and Discriminative Models.
Generative models are a class of machine learning algorithms that can generate new data samples that are similar to the training data. These models learn the underlying distribution of the input data and can produce realistic synthetic examples. In contrast, discriminative models are trained to predict a target variable based on input features. They focus on identifying patterns in the input data rather than generating new data.
Generative models can be further divided into two categories: generative adversarial networks (GANs) and variational autoencoders (VAEs). GANs consist


## RAG

### Loading

In [5]:

pdf_file_path = os.path.join(parent_dir, 'data', 'Applied Generative AI for Beginners.pdf')

loader = PyPDFLoader(pdf_file_path)
data = loader.load()

### Chunking

In [6]:
recursive_char_text_splitter=RecursiveCharacterTextSplitter(
                                                chunk_size=1000,
                                                chunk_overlap=50)
documents=recursive_char_text_splitter.split_documents(data)
print('documents length - ', len(documents))

documents length -  435


### Embeddings

In [7]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir, os.pardir))
embedding_path = os.path.join(parent_dir, 'model', 'embeddings')

embeddings=HuggingFaceEmbeddings(
    model_name='sentence-transformers/all-MiniLM-L6-v2', 
    model_kwargs={'device':'cpu'},
    cache_folder=embedding_path
)

c:\Users\LENOVO\sagar\work\AI_ML\gitrepo\GENAI\YOUTUBES\GenAI_research\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


### Vector store

In [8]:
vector_db_directory = os.path.join(parent_dir, 'model', 'vectorDB')
new_knowledge_base =FAISS.from_documents(documents, embeddings)
new_knowledge_base.save_local(vector_db_directory)


loaded_vector_db = FAISS.load_local(vector_db_directory, embeddings, allow_dangerous_deserialization=True)

In [9]:
retriever = loaded_vector_db.as_retriever()
retriever.get_relevant_documents(user_query)

c:\Users\LENOVO\sagar\work\AI_ML\gitrepo\GENAI\YOUTUBES\GenAI_research\venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='So, What Is Generative AI?\nGenerative AI  refers to a branch of artificial intelligence  that focuses on\ncreating models and algorithms  capable of generating new , original content,\nsuch as images, text, music, and even videos. Unlike traditional AI models\nthat are trained to perform specific tasks, generative AI models aim to learn\nand mimic patterns from existing data to generate new , unique outputs.\nGenerative AI has a wide range of applications. For instance, in\ncomputer vision, generative models can generate realistic images, create\nvariations of existing images, or even complete missing parts of an image.\nIn natural language processing, generative models can be used for language\ntranslation, text synthesis, or even to create conversational agents that\nproduce humanlike responses. Beyond these examples, generative ai can\nperform art generation, data augmentation, and even generating synthetic\nmedical images for research and diagnosis. It’ s a

### Prompt Template

In [10]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)

### Chain

In [11]:
from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type='stuff',
            retriever=loaded_vector_db.as_retriever(search_kwargs={'k': 4}),
            return_source_documents=True,
            chain_type_kwargs={'prompt': prompt}
    )

In [12]:
result = chain({'query':user_query}, return_only_outputs=True)
print('result - ', result)
ans = result['result']
print(f"Answer:{ans}")

c:\Users\LENOVO\sagar\work\AI_ML\gitrepo\GENAI\YOUTUBES\GenAI_research\venv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
Number of tokens (747) exceeded maximum context length (200).
Number of tokens (748) exceeded maximum context length (200).
Number of tokens (749) exceeded maximum context length (200).
Number of tokens (750) exceeded maximum context length (200).
Number of tokens (751) exceeded maximum context length (200).
Number of tokens (752) exceeded maximum context length (200).
Number of tokens (753) exceeded maximum context length (200).
Number of tokens (754) exceeded maximum context length (200).
Number of tokens (755) exceeded maximum context length (200).
Number of tokens (756) exceeded maximum context length (200).
Number of tokens (757) exceeded maximum context length (200).
Number of tokens (758) e

result -  {'result': '\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n', 'source_documents': [Document(page_content='So, What Is Generative AI?\nGenerative AI  refers to a branch of artificial intelligence  that focuses on\ncreating models and algorithms  capable of generating new , original content,\nsuch as images, text, music, and even videos. Unlike traditional AI models\nthat are trained to perform specific tasks, generative AI models aim to learn\nand mimic patterns from existing data to generate new , unique outputs.\nGenerative AI has a wide range of applications. For instance, in\ncomputer vision, generative models can generate realistic images, create\nvariations of existing images, or even complete missing parts of an image.\nIn natural language processing, generative mo